In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import json

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
ds = load_dataset("stdt1/test-class") 
print(ds)

DatasetDict({
    train: Dataset({
        features: ['definition', 'type'],
        num_rows: 900
    })
})


In [5]:
print(ds["train"][2]) # Called train in huggingface, but is really test

{'definition': 'Trollhätte kraftverk, Sveriges till storleken tredje kraftverk ( 1954, efter Harsprånget och Kilforsen ), utnyttjar vattenkraften i Trollhättefallen med sammanlagt 253, 000 kW installerad turbineffekt vid c : a 31 m nettofallhöjd ( uttagbar effekt c : a 225, 000 kW ). Till kraftförvaltningen T. höra även Vargöns kraftverk och Lilla Edets kraftverk. — T. var det första av statens kraftverk, tillkommet genom beslut av 1906 års riksdag', 'type': 0}


In [44]:
with open('TREDJE_EXTRACTED.json', encoding = "utf-8") as f: # Upload edition
    d = json.load(f)

In [45]:
big = []
for ent in d:
    big.append({"id": ent["id"], "headword": ent["headword"], "description": ent["description"]})

In [50]:
print(big[0])

{'id': 'U2_E1', 'headword': ['djäfvul'], 'description': 'djäfvul, af lat. diabolus); dels och i synnerhet genom\natt a blef å icke blott i alla de fall, då det af\ngammalt var långt (t. ex. åt, af fnsv. at, ty. ass,\nlat. edi), utan äfven då gammalt kort a förlängdes\nföre vissa konsonantgrupper, nämligen ld, rd, rt (före\nvokal), ng (i svenskan, men icke i danskan) och nd\n(i danskan, men icke i svenskan), t. ex. hålla:\nfnsv. halda, ty. halten; hård: fnsv. hardher,\nty. hart; Mårten, af lat. Martinus; lång: da. och\nty. lang; da. hånd. sv. och ty. hand. Ett nytt långt\na-ljud erhöll man vid samma tid genom förlängning\naf kort a före en enda konsonant, t. ex. gata,\nsak, af fnsv. gata, sak. Vida längre än svenskan,\nsom ännu är ganska rik på a-ljud, har danskan gått\ni fråga om uppgifvandet af detta ljud. Redan under\nmedeltiden öfvergick nämligen a till æ, hvaraf sedan\ne, i alla svagt betonade stafvelser (t. ex. gade,\ngata), och numera är äfven starkt betonadt långt a\n(utom i gra

In [7]:
model_name = "KB/bert-base-swedish-cased" # KB-BERT
num_classes = 3 # other, location, person
max_length = 128
batch_size = 16
num_epochs = 4
learning_rate = 2e-5

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [9]:
class EntityClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super(EntityClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [10]:
model = EntityClassifier(model_name, num_classes).to(device)

In [11]:
state_dict = torch.load("best_entity_classifier_so_far.pth")
model.load_state_dict(state_dict)

<All keys matched successfully>

In [12]:
model.eval()

EntityClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(50325, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [13]:
def num_to_class(num):
    if num == 0:
        return "other"
    elif num == 1:
        return "location"
    elif num == 2:
        return "person"
    else:
        return "something went wrong"

In [14]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return num_to_class(preds.item())

In [51]:
li = []
for ent in tqdm(big):
    pred = predict_sentiment(ent["description"], model, tokenizer, device)
    li.append({"id": ent["id"], "headword": ent["headword"], "description": ent["description"], "type": pred})

100%|██████████| 216889/216889 [46:21<00:00, 77.98it/s] 


In [55]:
print(li[0])

{'id': 'U2_E1', 'headword': ['djäfvul'], 'description': 'djäfvul, af lat. diabolus); dels och i synnerhet genom\natt a blef å icke blott i alla de fall, då det af\ngammalt var långt (t. ex. åt, af fnsv. at, ty. ass,\nlat. edi), utan äfven då gammalt kort a förlängdes\nföre vissa konsonantgrupper, nämligen ld, rd, rt (före\nvokal), ng (i svenskan, men icke i danskan) och nd\n(i danskan, men icke i svenskan), t. ex. hålla:\nfnsv. halda, ty. halten; hård: fnsv. hardher,\nty. hart; Mårten, af lat. Martinus; lång: da. och\nty. lang; da. hånd. sv. och ty. hand. Ett nytt långt\na-ljud erhöll man vid samma tid genom förlängning\naf kort a före en enda konsonant, t. ex. gata,\nsak, af fnsv. gata, sak. Vida längre än svenskan,\nsom ännu är ganska rik på a-ljud, har danskan gått\ni fråga om uppgifvandet af detta ljud. Redan under\nmedeltiden öfvergick nämligen a till æ, hvaraf sedan\ne, i alla svagt betonade stafvelser (t. ex. gade,\ngata), och numera är äfven starkt betonadt långt a\n(utom i gra

In [56]:
with open('upplaga_2_classification.json', 'w', encoding='utf-8') as f:
    json.dump(li, f, ensure_ascii=False, indent=4)
    f.close()

In [15]:
with open('MANUAL_TEST_CLASS.json', encoding = "utf-8") as f:
    data_test_val = json.load(f)
    f.close()

In [16]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [17]:
tok_test_ds = TextClassificationDataset(ds["train"]["definition"], ds["train"]["type"], tokenizer, max_length)

test_dl = DataLoader(tok_test_ds, batch_size=batch_size)

In [18]:
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [19]:
accuracy, report = evaluate(model, test_dl, device)
print(f"Validation Accuracy: {accuracy:.4f}")
print(report)

Validation Accuracy: 0.9367
              precision    recall  f1-score   support

           0       0.94      0.89      0.91       330
           1       0.91      0.96      0.94       330
           2       0.97      0.97      0.97       240

    accuracy                           0.94       900
   macro avg       0.94      0.94      0.94       900
weighted avg       0.94      0.94      0.94       900

